<a href="https://colab.research.google.com/github/ettouilebouael/pdm-thesis/blob/main/domain_adaptation/FD001_domain_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install livelossplot --quiet

from google.colab import drive
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from skimage.restoration import denoise_wavelet
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
import random

drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Memoire/PdM_code")

from models import *
from torch_modules import *
from utils import *

random.seed(0)
torch.manual_seed(0) 
np.random.seed(0)


Mounted at /content/drive


In [58]:
features =['s_2', 's_3', 's_4', 's_7', 's_8', 's_9','s_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']

seq_len = 30
data_num_s = 1
data_num_t = 2
batch_size = 128
learning_rate = 1e-4
epochs = 15
dropout = 0
input_size = len(features)
n_lstm_layers = 3
n_hidden_lstm = 32

In [57]:
X_train_s, y_train_s = train_preprocessing(data_num_s,features, seq_len)
X_test_s, y_test_s = online_preprocessing(data_num_s,features, seq_len)

dataset_s = {
    'train': TurboFanDataset(X_train_s, y_train_s),
    'test': TurboFanDataset(X_test_s , y_test_s)}

dataloader_s = {
    'train': DataLoader(dataset['train'], shuffle=True, batch_size=batch_size),
     'test': DataLoader(dataset['test'], shuffle=False, batch_size=batch_size)}

In [59]:
X_train_t, y_train_t = train_preprocessing(data_num_t,features, seq_len)
X_test_t, y_test_t = online_preprocessing(data_num_t,features, seq_len)

dataset_t = {
    'train': TurboFanDataset(X_train_t, y_train_t),
    'test': TurboFanDataset(X_test_t , y_test_t)}

dataloader_t = {
    'train': DataLoader(dataset['train'], shuffle=True, batch_size=batch_size ),
     'test': DataLoader(dataset['test'], shuffle=False, batch_size=batch_size)}

In [60]:
encoder_s = Encoder(input_size, n_hidden_lstm, n_lstm_layers, dropout)
rul_predictor_s = RULPredictor(n_hidden_lstm, dropout)

In [61]:
encoder_s_state_dict = torch.load("/content/drive/MyDrive/Memoire/PdM_code/models/fd001/encoder_fd001_rmse_20.41266917843625", map_location=torch.device('cpu'))
rul_predictor_s_state_dict = torch.load("/content/drive/MyDrive/Memoire/PdM_code/models/fd001/rul_predictor_fd001_rmse_20.41266917843625", map_location=torch.device('cpu'))

In [64]:
encoder_s.load_state_dict(encoder_s_state_dict)
rul_predictor_s.load_state_dict(rul_predictor_s_state_dict)

<All keys matched successfully>

In [67]:
with torch.no_grad(): 
  X = torch.from_numpy(X_test_t).float().cpu()
  y = torch.from_numpy(y_test_t).float().cpu()
  h = encoder_s(X)
  y_hat = rul_predictor_s(h)

  torch.sqrt(torch.nn.functional.mse_loss(y,y_hat))

In [72]:
n_batches = min(len(dataset_s['train']) // batch_size + 1, len(dataset_t['train']) // batch_size + 1)
n_batches

139

In [73]:
class Discriminator(torch.nn.Module):


  def __init__(self, n_hidden_lstm,dropout):
    super().__init__()
    self.n_hidden_lstm = n_hidden_lstm
    self.fc1 = torch.nn.Linear(n_hidden_lstm, 64)
    self.fc2 = torch.nn.Linear(64, 32)
    self.fc3 = torch.nn.Linear(32, 16)
    self.output = torch.nn.Linear(16, 1)
    self.dropout = torch.nn.Dropout(p=dropout)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x - self.dropout(x)
    x = self.fc2(x)
    x = F.relu(x)
    x - self.dropout(x)
    x = self.fc3(x)
    x = F.relu(x)
    x - self.dropout(x)
    x = self.output(x)
    F.sigmoid(x)
    return x

In [76]:
discriminator = Discriminator(n_hidden_lstm, dropout)
encoder
device = "cuda" if torch.cuda.is_available() else "cpu"

encoder_s.to(device)
rul_predictor_s.to(device)
discriminator.to(device)


optimizer_ds = torch.optim.Adam(discriminator.parameters(), learning_rate)
optimizer_encoder = torch.optim.Adam(encoder_t.parameters(), learning_rate)

encoder_s.train()
rul_predictor_s.train()
discriminator.train()



NameError: ignored

In [ ]:

optimizer_ds = optim.SGD(discriminator.parameters(), lr, weight_decay=0.01)
optimizer_encoder = optim.SGD(encoder_t.parameters(), lr, weight_decay=0.01)
loss_fn_domain =  nn.CrossEntropyLoss()
loss_fn_target_encoder =  nn.CrossEntropyLoss()
discriminator.to(device)
encoder_s.to(device)
encoder_t.to(device)
encoder_s.eval()
features = encoder_s(X_test_tg)
y_hat = rul_predictor(features)
y = y_test_tg
rmse_t_val = rmse_fn(y_hat, y.reshape(-1,1))
rmse_t_val
encoder_s.train()

best_rmse = 100000

for epoch_idx in range(n_epochs):
  
    print(f'Epoch {epoch_idx+1:04d} / {n_epochs:04d}', end='\n=================\n')
    discriminator_acc = []
    loss_encoder_t = []

    for b in range(batches):
      
      discriminator.train()
      encoder_t.train()
      # === train discrimantor
      X_s = X_train_sc[:,b*batch_size:(b+1)*batch_size,:]
      X_t = X_train_tg[:,b*batch_size:(b+1)*batch_size,:]

      optimizer_ds.zero_grad()

      features_s = encoder_s(X_s)
      features_t = encoder_t(X_t)
      features = torch.cat((features_s, features_t), 0).cuda()

      labels_s = torch.ones(features_s.size(0)).long()
      labels_t = torch.zeros(features_t.size(0)).long()
      labels = torch.cat((labels_s, labels_t), 0 ).cuda()

      domain_pred = discriminator(features)
  

      domain_loss = loss_fn_domain(domain_pred, labels)

      domain_loss.backward()

      optimizer_ds.step()

      classes = (domain_pred > 0.5).float()[:,1]
      ds_acc_current = (classes == labels).float().mean()
      discriminator_acc.append(ds_acc_current.item())
      

      # === train encoder

      optimizer_ds.zero_grad()
      optimizer_encoder.zero_grad()

      features_t = encoder_t(X_t)

      pred_domain  = discriminator(features_t)

      labels = torch.ones(features_t.size(0)).long().cuda()

      loss_encoder_t_current = loss_fn_target_encoder(pred_domain, labels)

      loss_encoder_t_current.backward()

      optimizer_encoder.step()

      loss_encoder_t.append(loss_encoder_t_current.item())

    # === targer domain rul prediction evaluation
    with torch.no_grad():
      encoder_t.eval()

      features = encoder_t(X_test_tg)
      y_hat = rul_predictor(features)
      y = y_test_tg
      rmse_t_val = rmse_fn(y_hat, y.reshape(-1,1))
      
      if rmse_t_val.item() < best_rmse:
        best_rmse = rmse_t_val.item()
        torch.save(feature_extractor, "best_target_encoder")
        print(f"Best target encoder saved at epoch {epoch_idx} with rmse_val {rmse_t_val.item():.5f}")


      print(f'[{epoch_idx+1}/{n_epochs}] ' f' discriminator_acc: {np.mean(discriminator_acc):.4f}'
      f' encoder_loss: {np.mean(loss_encoder_t):.4f}' f' rmse_val: {rmse_t_val.item():.4f}'
                )


In [ ]:
def fit_adapt(encoder_s, rul_predictor_s, discriminator, dataloader_s, dataloader_t, epochs, learning_rate, data_num):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    liveloss = PlotLosses()
    encoder.to(device)
    rul_predictor.to(device)
    optimizer = torch.optim.Adam(list(encoder.parameters()) + list(discriminator.parameters()), lr=learning_rate,weight_decay=0)
    criterion = torch.nn.MSELoss()
    criterion_dicriminator = torch.nn.BCELoss()
    best_loss = 100
    for epoch in range(1, epochs+1):
        encoder.train()
        rul_predictor.train()
        train_loss = []
        eval_loss = []
        logs = {}
        for batch in dataloader['train']:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            h_hat = encoder(X)
            y_hat = rul_predictor(h_hat)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        encoder.eval()
        rul_predictor.eval()
        with torch.no_grad():
          for batch in dataloader['test']:
            X, y = batch
            X, y = X.to(device), y.to(device)
            h_hat = encoder(X)
            y_hat = rul_predictor(h_hat)
            loss = criterion(y_hat, y)
            eval_loss.append(loss.item())

            rmse_train = np.sqrt(np.mean(train_loss))
            rmse_val = np.sqrt(np.mean(eval_loss))

          logs["loss"] = rmse_train
          logs["val_loss"] = rmse_val

          liveloss.update(logs)
          liveloss.send()

          if epoch > 25 :
            if best_loss > rmse_val:
              best_loss = rmse_val
              torch.save(encoder.state_dict(), f"models/fd00{data_num}/encoder_fd00{data_num}_rmse_{rmse_val}")
              torch.save(rul_predictor.state_dict(), f"models/fd00{data_num}/rul_predictor_fd00{data_num}_rmse_{rmse_val}")
              print(f'best model saved with rmse = {rmse_val} at epoch {epoch}')
